In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

参考： 
・https://www.kaggle.com/kamalnaithani/ncaam-simple-model-copy 

・https://www.kaggle.com/junichiromorita/ncaam-2021-eda-english-japanese

・https://www.kaggle.com/junichiromorita/ncaam-2021-eda-english-japanese

・https://qiita.com/Derek/items/70dea0482c943c66c050

・『[Kaggleデータ分析入門](https://www.amazon.co.jp/Pythonで動かして学ぶ-Kaggleデータ分析入門-AI-TECHNOLOGY-篠田/dp/4798165239/ref=sr_1_1?dchild=1&qid=1615682817&s=books&sr=1-1&text=篠田+裕之)』（著：篠田 裕之）

・『[Kaggleで勝つデータ分析の技術](https://www.amazon.co.jp/Kaggleで勝つデータ分析の技術-門脇-大輔-ebook/dp/B07YTDBC3Z/ref=sr_1_1?__mk_ja_JP=カタカナ&crid=11EOG1XUISJRX&dchild=1&keywords=kaggleで勝つデータ分析の技術&qid=1615682901&s=books&sprefix=Kaggleで%2Cstripbooks%2C269&sr=1-1)』（著：門脇 大輔）

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import optuna
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from optuna.integration import _lightgbm_tuner as tuner

トータルの得失点、アシスト、リバウンド（OF,DF）、シュート率(FG,FG3,FT)、ターンオーバー、スチール、ブロック、ファール，チームのシード、ホーム地を追加

In [ ]:
PATH = "../input/ncaam-march-mania-2021/"

In [ ]:
RegularSeason_DetailedResults = pd.read_csv(PATH + "MRegularSeasonDetailedResults.csv")
Tourney_Detailed_Results = pd.read_csv(PATH + "MNCAATourneyDetailedResults.csv")

In [ ]:
RegularSeason_DetailedResults = pd.read_csv(PATH + "MRegularSeasonDetailedResults.csv")
Tourney_Detailed_Results = pd.read_csv(PATH + "MNCAATourneyDetailedResults.csv")

In [ ]:
test_df = pd.read_csv(PATH + "MSampleSubmissionStage1.csv")

In [ ]:
#IDからシーズン、チームIDを抽出。PredはチームIDが若い方が勝てば1、負けで0

test_df["Season"] = test_df["ID"].map(lambda x: int(x[:4]))
test_df["WTeamID"] = test_df["ID"].map(lambda x: int(x[5:9]))
test_df["LTeamID"] = test_df["ID"].map(lambda x: int(x[10:14]))
test_df

In [ ]:
#IDからシーズン、チームIDを抽出。PredはチームIDが若い方が勝てば1、負けで0

test_df["Season"] = test_df["ID"].map(lambda x: int(x[:4]))
test_df["WTeamID"] = test_df["ID"].map(lambda x: int(x[5:9]))
test_df["LTeamID"] = test_df["ID"].map(lambda x: int(x[10:14]))
test_df = test_df.drop(["ID","Pred"],axis=1)
test_df

In [ ]:
#トーナメントの結果

Tourney_Detailed_Results.head()

In [ ]:
#予測の際に与えられる変数のみにする
#各変数の相関を見たいため，得失点は残しておく
#テストデータは2015~2019年のため，2014年以前のデータを学習データとする

Tourney_Detailed_Results = Tourney_Detailed_Results[(Tourney_Detailed_Results["Season"] < 2015)][["Season","WTeamID","LTeamID","WScore","LScore"]]
Tourney_Detailed_Results["Result"] = 1
Tourney_Detailed_Results

In [ ]:
all_df = pd.concat((Tourney_Detailed_Results, test_df), sort=False).reset_index(drop=True)
all_df

In [ ]:
#レギュラーシーズンの結果

RegularSeason_DetailedResults.head()

In [ ]:
Tourney_Seeds = pd.read_csv(PATH + "MNCAATourneySeeds.csv")

In [ ]:
#両チームのシードを追加

all_df = pd.merge(all_df, Tourney_Seeds, left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"], how="left")
all_df.rename(columns={"Seed":"WinningSeed"}, inplace=True)
all_df = all_df.drop(["TeamID"], axis=1)

all_df = pd.merge(all_df, Tourney_Seeds, left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"], how="left")
all_df.rename(columns={"Seed":"LosingSeed"}, inplace=True)
all_df = all_df.drop(["TeamID"], axis=1)

all_df

In [ ]:
#シードから，ホームの区分の記号を抽出
#コピーしたデータフレームに代入すると出る警告があるが，実行結果に問題がないため，一旦保留

all_df["WinningHome_sym"] = all_df["WinningSeed"].str.extract('([A-Z])',expand=True)
all_df["LosingHome_sym"] = all_df["LosingSeed"].str.extract('([A-Z])',expand=True)

all_df

In [ ]:
MSeasons = pd.read_csv(PATH + "MSeasons.csv")

In [ ]:
MSeasons.head()

In [ ]:
#それぞれ，出現回数が少ない地域をotherとして代入

MSeasons.loc[(MSeasons["RegionW"]!="East"),"RegionW"] = "OtherW"
MSeasons.loc[(MSeasons["RegionX"].str.contains("^(?!.*(West|Midwest|Southeast|South)).*$")),"RegionX"] = "OtherX"
MSeasons.loc[(MSeasons["RegionY"].str.contains("^(?!.*(Midwest|Southeast|South)).*$")),"RegionY"] = "OtherY"
MSeasons.loc[(MSeasons["RegionZ"].str.contains("^(?!.*(West)).*$")),"RegionZ"] = "OtherZ"

MSeasons = MSeasons.drop(["DayZero"],axis=1)

In [ ]:
#ホームを表す記号(W,X,Y,Z)が示す，地域ごとの出現回数

fig, axes = plt.subplots(2, 2, figsize=(20, 10))
sns.countplot(ax=axes[0][0], data=MSeasons, x='RegionW')
sns.countplot(ax=axes[0][1], data=MSeasons, x='RegionX')
sns.countplot(ax=axes[1][0], data=MSeasons, x='RegionY')
sns.countplot(ax=axes[1][1], data=MSeasons, x='RegionZ')
plt.show()

In [ ]:
#地域ごとに割り振られた記号ごとに，シーズンと地域名の変数を持つデータフレームにする

Mseasons_w = MSeasons[["Season","RegionW"]].copy()
Mseasons_x = MSeasons[["Season","RegionX"]].copy()
Mseasons_y = MSeasons[["Season","RegionY"]].copy()
Mseasons_z = MSeasons[["Season","RegionZ"]].copy()

Mseasons_w.rename(columns={"RegionW":"Region"},inplace=True)
Mseasons_x.rename(columns={"RegionX":"Region"},inplace=True)
Mseasons_y.rename(columns={"RegionY":"Region"},inplace=True)
Mseasons_z.rename(columns={"RegionZ":"Region"},inplace=True)

Mseasons_w["sym"] = "W"
Mseasons_x["sym"] = "X"
Mseasons_y["sym"] = "Y"
Mseasons_z["sym"] = "Z"

Mseasons_home = pd.concat((Mseasons_w, Mseasons_x, Mseasons_y, Mseasons_z)).reset_index(drop=True)

In [ ]:
#対戦データに両チームのホームの地域を追加

all_df = pd.merge(all_df, Mseasons_home, left_on=["Season", "WinningHome_sym"], right_on=["Season", "sym"], how="left")
all_df = all_df.drop(["WinningHome_sym","sym"],axis=1)
all_df.rename(columns={"Region":"WinningRegion"}, inplace=True)

all_df = pd.merge(all_df, Mseasons_home, left_on=["Season", "LosingHome_sym"], right_on=["Season", "sym"], how="left")
all_df = all_df.drop(["LosingHome_sym","sym"],axis=1)
all_df.rename(columns={"Region":"LosingRegion"}, inplace=True)

In [ ]:
#シード番号を抽出

all_df["WinningSeed"] = all_df["WinningSeed"].str.extract("(\d+)",expand=True)
all_df["LosingSeed"] = all_df["LosingSeed"].str.extract("(\d+)", expand=True)

all_df

In [ ]:
RegularSeason_Detailed_Results = pd.read_csv(PATH + "MRegularSeasonDetailedResults.csv")

In [ ]:
RegularSeason_Detailed_Results.head()

In [ ]:
#レギュラーシーズンの試合の詳細のカラム

RegularSeason_Detailed_Results.columns

In [ ]:
#レギュラーシーズンの試合の詳細データフレームを，各試合でのチームの各種スコアに変換

W_columns = ["Season","WTeamID","WFGM","WFGA","WFGM3","WFGA3","WFTM","WFTA","WOR","WDR","WAst","WTO","WStl","WPF"]
L_columns = ["Season","LTeamID","LFGM","LFGA","LFGM3","LFGA3","LFTM","LFTA","LOR","LDR","LAst","LTO","LStl","LPF"]
columns_name = ["Season", "TeamID", "FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "PF"]

season_winning_team = RegularSeason_Detailed_Results[W_columns]
season_winning_team_count = RegularSeason_Detailed_Results[["Season","WTeamID","WFGA"]].copy()
season_winning_team_count = season_winning_team_count.groupby(["Season","WTeamID"]).count()
season_losing_team = RegularSeason_Detailed_Results[L_columns]
season_losing_team_count = RegularSeason_Detailed_Results[["Season","LTeamID","LFGA"]].copy()
season_losing_team_count = season_losing_team_count.groupby(["Season","LTeamID"]).count()
season_winning_team.columns = columns_name
season_winning_team_count.index.names = ["Season","TeamID"]
season_winning_team_count.columns = ["Wcount"]
season_losing_team.columns = columns_name
season_losing_team_count.index.names = ["Season","TeamID"]
season_losing_team_count.columns = ["Lcount"]

RegularSeason_Detailed_Results = pd.concat((season_winning_team, season_losing_team)).reset_index(drop=True)
RegularSeason_Detailed_Results = pd.merge(RegularSeason_Detailed_Results, season_winning_team_count, on=["Season","TeamID"], how="left")
RegularSeason_Detailed_Results = pd.merge(RegularSeason_Detailed_Results, season_losing_team_count, on=["Season","TeamID"], how="left")
RegularSeason_Detailed_Results

In [ ]:
#シーズン，チームごとのレギュラーシーズンの各種スコア合計

RegularSeason_Detailed_Results = RegularSeason_Detailed_Results.groupby(["Season","TeamID"]).sum()
RegularSeason_Detailed_Results

In [ ]:
#各種シュート率（フリースロー，2pシュート，3pシュート），勝率を追加

RegularSeason_Detailed_Results["FG_suc_ratio"] = RegularSeason_Detailed_Results["FGM"]/RegularSeason_Detailed_Results["FGA"]
RegularSeason_Detailed_Results["FG3_suc_ratio"] = RegularSeason_Detailed_Results["FGM3"]/RegularSeason_Detailed_Results["FGA3"]
RegularSeason_Detailed_Results["FT_suc_ratio"] = RegularSeason_Detailed_Results["FTM"]/RegularSeason_Detailed_Results["FTA"]
RegularSeason_Detailed_Results["W_rate"] = RegularSeason_Detailed_Results["Wcount"]/(RegularSeason_Detailed_Results["Wcount"] + RegularSeason_Detailed_Results["Lcount"])

RegularSeason_Detailed_Results = RegularSeason_Detailed_Results.drop(["Wcount","Lcount"], axis=1)
RegularSeason_Detailed_Results

In [ ]:
RegularSeason_Compact_Results = pd.read_csv(PATH + "MRegularSeasonCompactResults.csv")

In [ ]:
#RegularSeason_Compact_Resultsを勝敗関係なく、各試合のチームごとの得失点とシーズンに。

season_winning_team = RegularSeason_Compact_Results[["Season","WTeamID","WScore","LScore"]].copy()
season_losing_team = RegularSeason_Compact_Results[["Season","LTeamID","LScore","WScore"]].copy()
season_winning_team.rename(columns={"WTeamID":"TeamID","WScore":"Score","LScore":"LosingScore"}, inplace=True)
season_losing_team.rename(columns={"LTeamID":"TeamID","LScore":"Score","WScore":"LosingScore"}, inplace=True)
RegularSeason_Compact_Results = pd.concat((season_winning_team, season_losing_team),sort=False).reset_index(drop=True)

RegularSeason_Compact_Results.head()

In [ ]:
#各チームのシーズンごとのスコアの合計
#平均ではなく合計なのは試合数の情報を内包させるため

RegularSeason_Compact_Results_Final = RegularSeason_Compact_Results.groupby(["Season","TeamID"])[["Score","LosingScore"]].sum().reset_index()
RegularSeason_Compact_Results_Final.head()

In [ ]:
#レギュラーシーズンの各種合計スコアやシュート率のスコアを，トーナメントでの対戦データに追加

all_df = pd.merge(all_df, RegularSeason_Detailed_Results, left_on=["Season","WTeamID"], right_on=["Season", "TeamID"], how="left")
all_df.rename(columns={"OR":"TWOR","DR":"TWDR","Ast":"TWAst","TO":"TWTO","Stl":"TWStl","PF":"TWPF","FGA":"TWFGA","FGM":"TWFGM","FGA3":"TWFGA3","FGM3":"TWFGM3","FTA":"TWFTA","FTM":"TWFTM","FG_suc_ratio":"WFG_suc_ratio","FG3_suc_ratio":"WFG3_suc_ratio","FT_suc_ratio":"WFT_suc_ratio","W_rate":"WW_rate"}, inplace=True)
all_df = pd.merge(all_df, RegularSeason_Detailed_Results, left_on=["Season","LTeamID"], right_on=["Season", "TeamID"], how="left")
all_df.rename(columns={"OR":"TLOR","DR":"TLDR","Ast":"TLAst","TO":"TLTO","Stl":"TLStl","PF":"TLPF","FGA":"TLFGA","FGM":"TLFGM","FGA3":"TLFGA3","FGM3":"TLFGM3","FTA":"TLFTA","FTM":"TLFTM","FG_suc_ratio":"LFG_suc_ratio","FG3_suc_ratio":"LFG3_suc_ratio","FT_suc_ratio":"LFT_suc_ratio","W_rate":"LW_rate"}, inplace=True)

all_df = pd.merge(all_df, RegularSeason_Compact_Results_Final, left_on=["Season","WTeamID"], right_on=["Season","TeamID"], how="left")
all_df.rename(columns={"Score":"WScoreT","LosingScore":"WLosingScoreT"}, inplace=True)
all_df= all_df.drop(["TeamID"],axis=1)
all_df = pd.merge(all_df, RegularSeason_Compact_Results_Final, left_on=["Season","LTeamID"], right_on=["Season","TeamID"], how="left")
all_df.rename(columns={"Score":"LScoreT","LosingScore":"LLosingScoreT"}, inplace=True)
all_df = all_df.drop(["TeamID"],axis=1)

all_df.head()

In [ ]:
#シードを数値型にする

all_df["WinningSeed"] = all_df["WinningSeed"].astype(int)
all_df["LosingSeed"] = all_df["LosingSeed"].astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
#ホームの地域をカテゴリ変数にする

categories = all_df.columns[all_df.dtypes == "object"]

for cat in categories:
    le = LabelEncoder()
    print(cat)
    if all_df[cat].dtypes == "object":
        le = le.fit(all_df[cat])
        all_df[cat] = le.transform(all_df[cat])

In [ ]:
all_df[categories] = all_df[categories].astype("category")

In [ ]:
#カラム名を変更
#変数"Result"に勝敗のデータを入れる（0->Seed2のチームが勝利，1->Seed1のチームが勝利）

columns_name_win = {"WinningSeed":"Seed1",
                "WinningRegion":"Region1",
                "TWOR":"TOR1",
                "TWDR":"TDR1",
                "TWAst":"TAst1",
                "TWTO":"TTO1",
                "TWStl":"TStl1",
                "TWPF":"TPF1",
                "TWFGA":"TFGA1",
                "TWFGM":"TFGM1",
                "TWFGA3":"TFGA31",
                "TWFGM3":"TFGM31",
                "TWFTA":"TFTA1",
                "TWFTM":"TFTM1",
                "WFG_suc_ratio":"FG_suc_ratio1",
                "WFG3_suc_ratio":"FG3_suc_ratio1",
                "WFT_suc_ratio":"FT_suc_ratio1",
                "WTeamID":"TeamID1",
                "WScore":"Score1",
                "WW_rate":"W_rate1",
                "WScoreT":"ScoreT1",
                "WLosingScoreT":"LosingScoreT1",
                "LosingSeed":"Seed2",
                "LosingRegion":"Region2",
                "TLOR":"TOR2",
                "TLDR":"TDR2",
                "TLAst":"TAst2",
                "TLTO":"TTO2",
                "TLStl":"TStl2",
                "TLPF":"TPF2",
                "TLFGA":"TFGA2",
                "TLFGM":"TFGM2",
                "TLFGA3":"TFGA32",
                "TLFGM3":"TFGM32",
                "TLFTA":"TFTA2",
                "TLFTM":"TFTM2",
                "LFG_suc_ratio":"FG_suc_ratio2",
                "LFG3_suc_ratio":"FG3_suc_ratio2",
                "LFT_suc_ratio":"FT_suc_ratio2",
                "LTeamID":"TeamID2",
                "LScore":"Score2",
                "LW_rate":"W_rate2",
                "LScoreT":"ScoreT2",
                "LLosingScoreT":"LosingScoreT2"
               }

all_df_win = all_df.copy()
all_df_win.rename(columns=columns_name_win,inplace=True)

In [ ]:
all_df_win

In [ ]:
all_df_win.columns

In [ ]:
#各種スコアを入れ替えたデータフレームを追加

columns_name_lose = {"WinningSeed":"Seed2",
                "WinningRegion":"Region2",
                "TWOR":"TOR2",
                "TWDR":"TDR2",
                "TWAst":"TAst2",
                "TWTO":"TTO2",
                "TWStl":"TStl2",
                "TWPF":"TPF2",
                "TWFGA":"TFGA2",
                "TWFGM":"TFGM2",
                "TWFGA3":"TFGA32",
                "TWFGM3":"TFGM32",
                "TWFTA":"TFTA2",
                "TWFTM":"TFTM2",
                "WFG_suc_ratio":"FG_suc_ratio2",
                "WFG3_suc_ratio":"FG3_suc_ratio2",
                "WFT_suc_ratio":"FT_suc_ratio2",
                "WTeamID":"TeamID2",
                "WScore":"Score2",
                "WW_rate":"W_rate2",
                "WScoreT":"ScoreT2",
                "WLosingScoreT":"LosingScoreT2",
                "LosingSeed":"Seed1",
                "LosingRegion":"Region1",
                "TLOR":"TOR1",
                "TLDR":"TDR1",
                "TLAst":"TAst1",
                "TLTO":"TTO1",
                "TLStl":"TStl1",
                "TLPF":"TPF1",
                "TLPF":"TPF1",
                "TLFGA":"TFGA1",
                "TLFGM":"TFGM1",
                "TLFGA3":"TFGA31",
                "TLFGM3":"TFGM31",
                "TLFTA":"TFTA1",
                "TLFTM":"TFTM1",
                "LFG_suc_ratio":"FG_suc_ratio1",
                "LFG3_suc_ratio":"FG3_suc_ratio1",
                "LFT_suc_ratio":"FT_suc_ratio1",
                "ScoreT1":"ScoreT2",
                "ScoreT2":"ScoreT1",
                "LTeamID":"TeamID1",
                "LScore":"Score1",
                "LW_rate":"W_rate1",
                "LScoreT":"ScoreT1",
                "LLosingScoreT":"LosingScoreT1",
               }

all_df_lose = all_df[(~all_df["Result"].isnull())].copy()
all_df_lose.rename(columns=columns_name_lose,inplace=True)
all_df_lose["Result"] = 0

In [ ]:
all_df_lose

In [ ]:
all_df_lose.columns

In [ ]:
#上の2つのデータフレームを結合
#チームを入れ替えた対戦データをResultが0（TeamID1が負け）の対戦として追加した

all_df = pd.concat((all_df_win,all_df_lose),sort=False).reset_index(drop=True)
all_df.head()

In [ ]:
#一度，この時点でLightGBMで学習させた（クロスバリエーション）

#各種スコアの差を取らなかった際の結果 -> 0.58767
#各種スコアの差を取った際の結果 -> 0.58363


#各種スコアの差をとる

diff_columns = ["Seed","TOR","TDR","TAst","TTO","TStl","TPF","FG_suc_ratio","FG3_suc_ratio","FT_suc_ratio","ScoreT","LosingScoreT","W_rate","Score"]

for col in diff_columns:
    all_df[col+"_diff"] = all_df[col+"1"] - all_df[col+"2"]

all_df.head()

◆ 追加していない指標を追加する

・試合時点で使用可能なランキング（精度が良いランキングの差の合計のみ追加すれば良いかも）

・両チームのコーチの功績（最初はコーチをした回数のみにしてみる）

・所属会議の追加

・Division-1になった年と外れた年

得失点差と相関が見られないデータは削除しておく

◆ 試すこと

近年のデータでのモデルとのアンサンブル（まだ）

XGBoostでの学習

時系列に沿ったクロスバリエーション（まだ）

In [ ]:
Cities = pd.read_csv(PATH + "Cities.csv")
Conference = pd.read_csv(PATH + "Conferences.csv")
MConferenceTourneyGames = pd.read_csv(PATH + "MConferenceTourneyGames.csv")
MGameCities = pd.read_csv(PATH + "MGameCities.csv")
MMasseyOrdinals = pd.read_csv(PATH + "MMasseyOrdinals.csv")
MNCAATourneySeedRoundSlots = pd.read_csv(PATH + "MNCAATourneySeedRoundSlots.csv")
MNCAATourneySlots = pd.read_csv(PATH + "MNCAATourneySlots.csv")
MSecondaryTourneyCompactResults = pd.read_csv(PATH + "MSecondaryTourneyCompactResults.csv")
MSecondaryTourneyTeams = pd.read_csv(PATH + "MSecondaryTourneyTeams.csv")
MTeamCoaches = pd.read_csv(PATH + "MTeamCoaches.csv")
MTeamConferences = pd.read_csv(PATH + "MTeamConferences.csv")
MTeamSpellings = pd.read_csv(PATH + "MTeamSpellings.csv", encoding="shift-jis")
MTeams = pd.read_csv(PATH + "MTeams.csv")

In [ ]:
#都市の州，地域を対戦データに追加済みのため一旦保留

Cities.head()

In [ ]:
#会議の略語と名前．いらない．

Conference.head()

In [ ]:
#各会議が実施したトーナメントの勝敗

MConferenceTourneyGames.head()

In [ ]:
#全ての試合の勝敗，CRTypeでそれぞれ結果の勝敗を他のデータフレームから確認できる
#レギュラーシーズンの試合結果と合わせて，NCAAMの結果を予測する指標にできそう

MGameCities.head()

In [ ]:
#各チームのランキングと予測に使えるようになる日
#確実にトーナメントの予測に使えるランキングシステムを使う

MMasseyOrdinals.head()

In [ ]:
#一つのチームの一つのシーズンにランキングがいくつもある

MMasseyOrdinals.groupby(["Season","TeamID"]).count()

In [ ]:
#一つのチームの一つのシーズンに同じランキングシステムが何度もランキングを付け直している
#RankingDayNumがそれぞれ一番大きい時のOrdinalRankを使えば良いはず

MMasseyOrdinals.groupby(["Season","TeamID","SystemName"]).count()

In [ ]:
#どのランキングも遅くて133日目にわかる
#トーナメントは134日目以降の試合しかないため，ランキングは全て使用可能と見ていい

MMasseyOrdinals["RankingDayNum"].describe()

In [ ]:
#シーズン，チームごとの順位の平均を出す
#RankingDayNumが100以上のランキングのみ使用すると，若干スコアが改善
#チームごとに出るランキングの種類数が違うので合計ではなく平均
#調べた結果いくつか精度の高いランキングがあるようなので，別途変数におこす

Rank_columns = ["MOR","POM"]

New_Ordinals_tmp = MMasseyOrdinals[(MMasseyOrdinals["RankingDayNum"] >= 100)].groupby(["Season","TeamID","SystemName"]).last()
New_Ordinals_tmp = New_Ordinals_tmp.unstack(level="SystemName")
New_Ordinals = pd.DataFrame(New_Ordinals_tmp[["OrdinalRank"]].mean(axis=1))

for col in Rank_columns:
    New_Ordinals[col] = New_Ordinals_tmp[[("OrdinalRank",col)]]
    
New_Ordinals.rename(columns={0:"OrdinalRank"},inplace=True)
New_Ordinals

In [ ]:
#対戦データに順位の合計を追加

all_df = pd.merge(all_df, New_Ordinals, left_on=["Season","TeamID1"], right_on=["Season","TeamID"], how="left")
all_df.rename(columns={"OrdinalRank":"OrdinalRank1"}, inplace=True)

all_df = pd.merge(all_df, New_Ordinals, left_on=["Season","TeamID2"], right_on=["Season","TeamID"], how="left")
all_df.rename(columns={"OrdinalRank":"OrdinalRank2"}, inplace=True)

for col in Rank_columns:
    all_df.rename(columns={col+"_x":col+"1"}, inplace=True)
    all_df.rename(columns={col+"_y":col+"2"}, inplace=True)

In [ ]:
#順位の差をとる

all_df["OrdinalRank_diff"] = all_df["OrdinalRank1"] - all_df["OrdinalRank2"]

for col in Rank_columns:
    all_df[col+"_diff"] = all_df[col+"1"] - all_df[col+"2"]

In [ ]:
all_df.head()

In [ ]:
#シード,ゲームラウンドごとの最速の日程と一番遅い日程
#いらなさそう

MNCAATourneySeedRoundSlots.head()

In [ ]:
#シーズン，ゲームスロットごとのシードの組み合わせ
#いらなさそう

MNCAATourneySlots.head()

In [ ]:
#NCAAMと同時進行される他のトーナメント
#基本的にチームメンバーの主力は毎年入れ替わるはずだから，結果を予測する対戦より前の年に行われた対戦は入力値に使わないでおく

MSecondaryTourneyCompactResults.head()

In [ ]:
MSecondaryTourneyTeams.head()

In [ ]:
MTeamCoaches.head()

In [ ]:
MTeamCoaches.describe()

In [ ]:
#コーチがSeasonまでに何度コーチをしてきたかを変数として追加

MTeamCoaches["Coaching_count"] = np.nan

for y in range(1985,2021):
    Coaches_counts = pd.DataFrame(MTeamCoaches[(MTeamCoaches["Season"]<y)]["CoachName"].value_counts())
    
    Coaches_counts.index.names = ["CoachName"]
    Coaches_counts.columns = ["Coaching_count_tmp"]
    
    Coaches_counts["Season"] = y
    MTeamCoaches = pd.merge(MTeamCoaches, Coaches_counts, on=["Season","CoachName"], how="left")
    
    MTeamCoaches.loc[(MTeamCoaches["Coaching_count"].isnull()),"Coaching_count"] = MTeamCoaches["Coaching_count_tmp"]
    MTeamCoaches = MTeamCoaches.drop(["Coaching_count_tmp"], axis=1)

MTeamCoaches = MTeamCoaches.fillna(0)

In [ ]:
MTeamCoaches.head()

In [ ]:
MTeamCoaches.groupby(["Season","TeamID"]).count().describe()

In [ ]:
MTeamCoaches_max = MTeamCoaches[["Season","TeamID","LastDayNum"]].groupby(["Season","TeamID"]).max()
MTeamCoaches_max.columns = ["MaxLastDayNum"]
MTeamCoaches = pd.merge(MTeamCoaches, MTeamCoaches_max, on=["Season","TeamID"], how="left")
MTeamCoaches = MTeamCoaches[(MTeamCoaches["LastDayNum"] == MTeamCoaches["MaxLastDayNum"])]

In [ ]:
MTeamCoaches = MTeamCoaches.drop(["MaxLastDayNum"],axis=1)
MTeamCoaches.groupby(["Season","TeamID"]).count().describe()

In [ ]:
#両チームのコーチの今までのコーチを経験した回数

all_df = pd.merge(all_df, MTeamCoaches.drop(["CoachName"],axis=1), left_on=["Season","TeamID1"], right_on=["Season","TeamID"], how="left")
all_df.rename(columns={"Coaching_count":"Coaching_count1","FirstDayNum":"CoachFirstDayNum1","LastDayNum":"CoachLastDayNum1"},inplace=True)
all_df = all_df.drop(["TeamID"],axis=1)

all_df = pd.merge(all_df, MTeamCoaches.drop(["CoachName"],axis=1), left_on=["Season","TeamID2"], right_on=["Season","TeamID"], how="left")
all_df.rename(columns={"Coaching_count":"Coaching_count2","FirstDayNum":"CoachFirstDayNum2","LastDayNum":"CoachLastDayNum2"},inplace=True)
all_df = all_df.drop(["TeamID"],axis=1)

all_df["Coaching_count_diff"] = all_df["Coaching_count1"] - all_df["Coaching_count2"]
all_df = all_df.drop(["Coaching_count1","Coaching_count2"],axis=1)

all_df

In [ ]:
#シーズン,チームごとの会議

MTeamConferences.head()

In [ ]:
#会議の数が多すぎてカテゴリ変数で対戦データに追加しても大きな役割は見込めない（と思われる）
#追加してみて重要度を確認後，必要に応じて各会議のチームの戦績を追加

MTeamConferences["ConfAbbrev"].value_counts()

In [ ]:
MTeamConferences

In [ ]:
#各会議ごとの1シーズンのチームの数

MTeamConferences_count = MTeamConferences[["Season","TeamID","ConfAbbrev"]].groupby(["Season","ConfAbbrev"]).count().unstack()
MTeamConferences_count.describe()

In [ ]:
MTeamConferences.columns

In [ ]:
categories = MTeamConferences.columns[(MTeamConferences.dtypes=="object")]

for cat in categories:
    le = LabelEncoder()
    le = le.fit(MTeamConferences[cat])
    MTeamConferences[cat] = le.transform(MTeamConferences[cat])

In [ ]:
all_df = pd.merge(all_df, MTeamConferences, left_on=["Season","TeamID1"], right_on=["Season","TeamID"], how="left")
all_df = all_df.drop(["TeamID"],axis=1)
all_df.rename(columns={"ConfAbbrev":"ConfAbbrev1"}, inplace=True)

all_df = pd.merge(all_df, MTeamConferences, left_on=["Season","TeamID2"], right_on=["Season","TeamID"], how="left")
all_df = all_df.drop(["TeamID"],axis=1)
all_df.rename(columns={"ConfAbbrev":"ConfAbbrev2"}, inplace=True)

In [ ]:
all_df.head()

In [ ]:
#チーム名のスペル．いらない

MTeamSpellings.head()

In [ ]:
#Division-1になった年と外れた年
#試合時点でDivision-1であることは分かっているので，対戦データにFirstD1Seasonのみを追加し，その差をとる

MTeams = MTeams.drop(["TeamName"],axis=1)
MTeams.describe()

In [ ]:
all_df = pd.merge(all_df, MTeams, left_on=["TeamID1"], right_on=["TeamID"], how="left")
all_df = all_df.drop(["TeamID","LastD1Season"],axis=1)
all_df.rename(columns={"FirstD1Season":"FirstD1Season1"},inplace=True)

all_df = pd.merge(all_df, MTeams, left_on=["TeamID2"], right_on=["TeamID"], how="left")
all_df = all_df.drop(["TeamID","LastD1Season"],axis=1)
all_df.rename(columns={"FirstD1Season":"FirstD1Season2"},inplace=True)

all_df["FirstD1Season_diff"] = all_df["FirstD1Season1"] - all_df["FirstD1Season2"]
all_df = all_df.drop(["FirstD1Season1","FirstD1Season2"],axis=1)


In [ ]:
all_df[["ConfAbbrev1","ConfAbbrev2"]] = all_df[["ConfAbbrev1","ConfAbbrev2"]].astype("category")

In [ ]:
#指標の分布を確認
#差を取った変数はどれも正規分布であるため非線形変換の必要性はないと思われる

for col in all_df.columns[all_df.dtypes != "category"]:
    fig, axes = plt.subplots(1, 1, figsize=(20, 5))
    sns.histplot(ax=axes, data=all_df, x=col)
    plt.show()

In [ ]:
all_df.columns

In [ ]:
all_df_corr = all_df.corr()
plt.figure(figsize=(50,50))
sns.heatmap(all_df_corr,-1,1,annot=True)

In [ ]:
#OrdinalRankとScoreTとSeedとW_rate以外，ほとんど相関がないように見える

hide_columns = ["Season","TeamID1","TeamID2","Score1","Score2","Result","Score_diff"]

for col1 in all_df.columns:
    if not col1 in hide_columns:
        all_df.plot.scatter(x=col1,y="Score_diff",alpha=0.8)

得失点差とある程度の相関があるように見える変数

Seed1,TFGM1,TFGA1,TOR1,TDR1,TAst1,FG_suc_ratio1,W_rate1,ScoreT1
Seed2,TFGM2,TFGA2,TOR2,TDR2,TAst2,FG_suc_ratio2,W_rate2,ScoreT2
Seed_diff,TOR_diff,TDR_diff,TAst_diff,TStl_diff,FG_suc_ratio_diff,ScoreT_diff,W_rate_diff,OrdinalRank_diff,Coaching_count_diff,FirstD1Season_diff,MOR_diff,POM_diff

差を取った変数のみで十分だと思われる

In [ ]:
train_columns = ["Seed_diff","TOR_diff","TDR_diff","TAst_diff","TStl_diff","FG_suc_ratio_diff","ScoreT_diff","W_rate_diff","OrdinalRank_diff","Coaching_count_diff","FirstD1Season_diff","MOR_diff","POM_diff"]

X = all_df[(~all_df["Result"].isnull())][train_columns]
y = all_df[(~all_df["Result"].isnull())].Result
test_df = all_df[(all_df["Result"].isnull())][train_columns]

In [ ]:
lgb_num_leaves_max = 255
lgb_in_leaf = 50
lgb_lr = 0.0001
lgb_bagging = 7

In [ ]:
params_lgb = {"num_leaves": lgb_num_leaves_max,
             "min_data_in_leaf":lgb_in_leaf,
             "objective":"binary",
             "max_depth":-1,
             "learning_rate":lgb_lr,
             "boosting_type":"gbdt",
             "bagging_seed":lgb_bagging,
             "metric":"logloss",
             "verbosity":-1,
             "random_state": 42,
             }

In [ ]:
#LightGBMで学習（クロスバリエーション）
#カテゴリ変数の設定のところで警告が出ているが，原因は不明
NFOLDS = 10
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds_lgb = np.zeros(test_df.shape[0])
y_train_lgb = np.zeros(X.shape[0])
y_oof = np.zeros(X.shape[0])

feature_importances = pd.DataFrame()
feature_importances["feature"] = columns

for fold_n, (train_index, valid_index) in enumerate(splits):
    print("Fold:", fold_n+1)
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)
    
    clf = lgb.train(params_lgb, dtrain, num_boost_round=10000, valid_sets = [dtrain, dvalid], verbose_eval=200)
    importance = pd.DataFrame(clf.feature_importance(), index=X_train.columns, columns=["importance"]).sort_values(by="importance",ascending=True)
    importance.plot.barh()
    
    feature_importances[f'fold_{fold_n +1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    
    y_train_lgb += clf.predict(X) / NFOLDS
    y_preds_lgb += clf.predict(test_df) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()

In [ ]:
# Score -> 

submission_df = pd.read_csv(PATH + "MSampleSubmissionStage1.csv")
y_preds = y_preds_lgb
submission_df["Pred"] = y_preds
submission_df.to_csv("submission01.csv", index=False)

・ランキングはやっぱり強い

・値がほとんど変わらない変数の重要度が低くなっている

In [ ]:
#結果にほぼ影響しない変数を削除
#そもそもどのデータもほとんど同じ値だったと考えられる
#コーチの日にちの変数は途中でコーチが変わったりしたかの変数に変えれば良いかも

all_df = all_df.drop(["FirstD1Season_diff"],axis=1)

Optunaでハイパーパラメータを最適化

In [ ]:
train_columns = ["Seed_diff","TOR_diff","TDR_diff","TAst_diff","TStl_diff","FG_suc_ratio_diff","ScoreT_diff","W_rate_diff","OrdinalRank_diff","Coaching_count_diff","MOR_diff","POM_diff"]

X = all_df[(~all_df["Result"].isnull())][train_columns]
y = all_df[(~all_df["Result"].isnull())].Result
test_df = all_df[(all_df["Result"].isnull())][train_columns]

In [ ]:
params_lgb = {"objective":"binary",
             "metric":"binary_logloss"}

In [ ]:
#Optunaでのハイパーパラメータ最適化．
#色々した結果，チューナーをそのまま使うのが一番精度が高い

NFOLDS = 10
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds_lgb = np.zeros(test_df.shape[0])
y_train_lgb = np.zeros(X.shape[0])
y_oof = np.zeros(X.shape[0])
#categories = ["Region1","Region2","ConfAbbrev1","ConfAbbrev2"]

feature_importances = pd.DataFrame()
feature_importances["feature"] = columns

for fold_n, (train_index, valid_index) in enumerate(splits):
    print("Fold:", fold_n+1)
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)
    
    clf = tuner.train(params_lgb,
                    dtrain,
                    valid_sets=[dtrain, dvalid],
                    num_boost_round=50000,
                    early_stopping_rounds=500,
                    verbose_eval=50)
    
    importance = pd.DataFrame(clf.feature_importance(), index=X_train.columns, columns=["importance"]).sort_values(by="importance",ascending=True)
    importance.plot.barh()
    
    feature_importances[f'fold_{fold_n +1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    
    y_train_lgb += clf.predict(X) / NFOLDS
    y_preds_lgb += clf.predict(test_df) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()

In [ ]:
#スコア -> 0.53814

submission_df = pd.read_csv(PATH + "MSampleSubmissionStage1.csv")
y_preds = y_preds_lgb
submission_df["Pred"] = y_preds
submission_df.to_csv("submission02.csv", index=False)

XGBoostで学習

In [ ]:
train_columns = ["Seed_diff","TOR_diff","TDR_diff","TAst_diff","TStl_diff","FG_suc_ratio_diff","ScoreT_diff","W_rate_diff","OrdinalRank_diff","Coaching_count_diff","MOR_diff","POM_diff"]

X = all_df[(~all_df["Result"].isnull())][train_columns]
y = all_df[(~all_df["Result"].isnull())].Result
test_df = all_df[(all_df["Result"].isnull())][train_columns]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1234, shuffle=False, stratify=None)

In [ ]:
def logloss(valid, pred, eps=1e-15):
    p = np.clip(pred, eps, 1-eps)
    logloss_valid_df = pd.DataFrame(valid)
    logloss_pred_df = pd.DataFrame(p)
    logloss_df = pd.concat((logloss_valid_df, logloss_pred_df), axis=1)
    logloss_df.columns = ["valid","pred"]
    logloss_df.loc[(logloss_df["valid"]==0),"pred"] = 1 - logloss_df["pred"]
    
    return -np.log(logloss_df["pred"]).mean()

In [ ]:
def objective(trial):
    xgb_params = {"objective":"binary:logistic",
                  "learning_rate":0.0001,
                  "seed":1234,
                  "eval_metric":"logloss",
                  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
                  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
                  "min_child_weight":trial.suggest_uniform("min_child_weight",0.2,2.0),
                  "max_depth":trial.suggest_int("max_depth",10,20),
                  "colsample_bytree":trial.suggest_uniform("colsample_bytree",0.4,0.99),
                  "subsample":trial.suggest_uniform("subsample",0.2,0.99),
                  "lambda":trial.suggest_uniform("lambda",0.1,2.0)
                 }
    
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_eval = xgb.DMatrix(X_valid, label=y_valid)
    evals = [(xgb_train, "train"), (xgb_eval, "eval")]
    
    model_xgb = xgb.train(xgb_params, xgb_train,
                         evals=evals,
                         num_boost_round=20000,
                         early_stopping_rounds=200,
                         verbose_eval=100)
    
    y_pred = model_xgb.predict(xgb_eval)
    score = logloss(y_valid, y_pred)
    
    return score

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=500)
study.best_params

In [ ]:
params_xgb = {
    "objective":"binary:logistic",
    "learning_rate":0.0001,
    "seed":1234,
    "eval_metric":"logloss",
    'reg_lambda': 0.5313661324133436,
 'reg_alpha': 56.043547449902384,
 'min_child_weight': 1.4353416333650515,
 'max_depth': 18,
 'colsample_bytree': 0.9133588762737089,
 #'subsample': 0.6804555526846559,
 #'lambda': 0.7632659497765832
}

In [ ]:
#XGBoostで学習（クロスバリエーション）

NFOLDS = 10
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds_xgb = np.zeros(test_df.shape[0])
y_train_xgb = np.zeros(X.shape[0])
y_oof = np.zeros(X.shape[0])

feature_importances = pd.DataFrame()
feature_importances["feature"] = columns

xgb_X = xgb.DMatrix(X, label=y)
xgb_test = xgb.DMatrix(test_df)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print("Fold:", fold_n+1)
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_valid = xgb.DMatrix(X_valid, label=y_valid)
    evals = [(xgb_train, "train"), (xgb_valid, "eval")]
    
    model_xgb = xgb.train(params_xgb,
                          xgb_train,
                          evals=evals,
                          num_boost_round=20000,
                          early_stopping_rounds=200,
                          verbose_eval=200)
    
    y_pred_valid = model_xgb.predict(xgb_valid)
    y_oof[valid_index] = y_pred_valid
    
    y_train_xgb += model_xgb.predict(xgb_X) / NFOLDS
    y_preds_xgb += model_xgb.predict(xgb_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()

In [ ]:
#スコア -> 0.64211
#どこか間違っている気がする

submission_df = pd.read_csv(PATH + "MSampleSubmissionStage1.csv")
y_preds = y_preds_xgb
submission_df["Pred"] = y_preds
submission_df.to_csv("submission03.csv", index=False)